# Лекция 6

# Решение систем линейных уравнений. LU разложение. Число обусловленности

## На прошлой лекции

- Умножение матрицы на вектор (матвек)
- Матричное умножение
- Иерархия памяти
- BLAS: структура и реализации

## План на сегодня

- Завершаем тему прошлой лекции
    - Метод Штрассена и его сложность
    - Связь методы Штрассена и тензорного разложения
    - RL подход для получения новых алгоритмов матричного умножения
- Линейные системы
- (P)LU разложение
- Устойчивость решения линейных систем и число обусловленности

## Улучшение алгоритмов матричного умножения

- Наивная реализация $O(n^3)$
- Можно ли сократить до $O(n^2)$ ?

**A**: алгоритм умножения матриц со сложностью $\mathcal{O}(n^2)$ до сих пор не найден.

* Метод Штрассена – $\mathcal{O}(n^{2.807\dots})$, иногда используется на практике

* [Текущий мировой рекорд](http://arxiv.org/pdf/1401.7714v1.pdf) $\mathcal{O}(n^{2.37\dots})$ – большая константа, не практичный, основан на [алгоритме Coppersmith-Winograd'a](https://en.wikipedia.org/wiki/Coppersmith%E2%80%93Winograd_algorithm).

Рассмотрим метод Штрассена более детально.

## Наивный алгоритм

Пусть $A$ и $B$ две матрицы $2\times 2$. Наивное умножение $C = AB$

$$
\begin{bmatrix} c_{11} & c_{12} \\ c_{21} & c_{22}  \end{bmatrix}  =
\begin{bmatrix} a_{11} & a_{12} \\ a_{21} & a_{22}  \end{bmatrix}
\begin{bmatrix} b_{11} & b_{12} \\ b_{21} & b_{22}  \end{bmatrix} =
\begin{bmatrix} 
a_{11}b_{11} + a_{12}b_{21} & a_{11}b_{21} + a_{12}b_{22} \\ 
a_{21}b_{11} + a_{22}b_{21} & a_{21}b_{21} + a_{22}b_{22} 
\end{bmatrix}
$$

состоит из $8$ умножений и $4$ сложений.

## Алгоритм Штрассена

В работе [Gaussian elimination is not optimal](http://link.springer.com/article/10.1007%2FBF02165411?LI=true) (1969 г.) Штрассен обнаружил, что можно вычислить матрицу $C$, используя 18 сложений и только 7 умножений:

$$
\begin{split}
c_{11} &= f_1 + f_4 - f_5 + f_7, \\
c_{12} &= f_3 + f_5, \\
c_{21} &= f_2 + f_4, \\
c_{22} &= f_1 - f_2 + f_3 + f_6,
\end{split}
$$
где

$$
\begin{split}
f_1 &= (a_{11} + a_{22}) (b_{11} + b_{22}), \\
f_2 &= (a_{21} + a_{22}) b_{11}, \\
f_3 &= a_{11} (b_{12} - b_{22}), \\
f_4 &= a_{22} (b_{21} - b_{11}), \\
f_5 &= (a_{11} + a_{12}) b_{22}, \\
f_6 &= (a_{21} - a_{11}) (b_{11} + b_{12}), \\
f_7 &= (a_{12} - a_{22}) (b_{21} + b_{22}).
\end{split}
$$

К счастью, эти формулы работают даже если $a_{ij}$ и $b_{ij}$, $i,j=1,2$ – блочные матрицы.

Таким образом, схема алгоритма Штрассена такова 
- Сначала мы  <font color='red'>разбиваем</font> матрицы $A$ и $B$ размера $n\times n$, $n=2^d$ <font color='red'> на 4 блока</font> размера $\frac{n}{2}\times \frac{n}{2}$
- После чего <font color='red'>вычисляем произведения</font> по указанным выше формулам <font color='red'>рекурсивно</font>

Это приводит нас к идее **разделяй и властвуй**.

## Сложность метода Штрассена

#### Число умножений

Обозначим $M(n)$ число умножений необходимых для вычисления произведения двух матриц $n\times n$ с помощью стратегии "разделяй и властвуй".
Тогда для наивного алгоритма получаем следующее число умножений

$$
M_\text{naive}(n) = 8 M_\text{naive}\left(\frac{n}{2} \right) = 8^2 M_\text{naive}\left(\frac{n}{4} \right)
= \dots = 8^{d-1} M(1) = 8^{d} = 8^{\log_2 n} = n^{\log_2 8} = n^3
$$

Итак, даже используя стратегию "разделяй и властвуй" мы не улучшаем сложность в $\mathcal{O}(n^3)$ операций.

Вычислим число умножений в методе Штрассена:

$$
M_\text{strassen}(n) = 7 M_\text{strassen}\left(\frac{n}{2} \right) = 7^2 M_\text{strassen}\left(\frac{n}{4} \right)
= \dots = 7^{d-1} M(1) = 7^{d} = 7^{\log_2 n} = n^{\log_2 7}
$$

#### Число сложений

Для алгоритма Штрассена:

$$
A_\text{strassen}(n) = 7 A_\text{strassen}\left( \frac{n}{2} \right) + 18 \left( \frac{n}{2} \right)^2
$$
поскольку на первом уровне нам нужно сложить матрицы размера $\frac{n}{2}\times \frac{n}{2}$ 18 раз, а потом идти на следующий уровень рекурсии для каждого из 7 умножений. 
Таким образом,

<font size=1.0>
$$
\begin{split}
A_\text{strassen}(n) =& 7 A_\text{strassen}\left( \frac{n}{2} \right) + 18 \left( \frac{n}{2} \right)^2 = 7 \left(7 A_\text{strassen}\left( \frac{n}{4} \right) + 18 \left( \frac{n}{4} \right)^2 \right) + 18 \left( \frac{n}{2} \right)^2 =
7^2 A_\text{strassen}\left( \frac{n}{4} \right) + 7\cdot 18 \left( \frac{n}{4} \right)^2 +  18 \left( \frac{n}{2} \right)^2 = \\
=& \dots = 18 \sum_{k=1}^d 7^{k-1} \left( \frac{n}{2^k} \right)^2 = \frac{18}{4} n^2 \sum_{k=1}^d \left(\frac{7}{4} \right)^{k-1} = \frac{18}{4} n^2 \frac{\left(\frac{7}{4} \right)^d - 1}{\frac{7}{4} - 1} = 6 n^2 \left( \left(\frac{7}{4} \right)^d - 1\right) \leqslant 6 n^2 \left(\frac{7}{4} \right)^d = 6 n^{\log_2 7}
\end{split}
$$
</font>

(так как $4^d = n^2$ и $7^d = n^{\log_2 7}$).


Асимптотику для $A(n)$ также можно найти с помощью следующей [теоремы](https://en.wikipedia.org/wiki/Master_theorem).

### Итоговая сложность метода Штрассена

Итоговая сложность –  $M_\text{strassen}(n) + A_\text{strassen}(n)=$ <font color='red'>$7 n^{\log_2 7}$</font>. 
Метод Штрассена становится быстрее наивного алгоритма, если

$$
\begin{split}
2n^3 &> 7 n^{\log_2 7}, \\
n &> 667,
\end{split}
$$

поэтому не самая хорошая идея спускаться на самый низкий уровень рекурсии.

## Является ли метод Штрассена практически важным?

- Долгое время считалось, что метод Штрассена имеет только теоретический интерес и плохо применим на практике
- Статья [Strassen algorithm reloaded](http://jianyuhuang.com/papers/sc16.pdf)
опровергает утверждение, что метод Штрассена не практичен, и предлагает реализацию, которая работает не только на больших матрицах.
- Метод Штрассена научились "выучивать" с помощью нейросетей
    - [V. Elser, 2016](https://www.jmlr.org/papers/volume17/16-074/16-074.pdf)
    - [M. Tschanne et al, 2018](https://arxiv.org/pdf/1712.03942.pdf)
    - [A. Fawzi, 2022](https://www.nature.com/articles/s41586-022-05172-4)

## Метод Штрассена и тензорное разложение

- Получение формул для алгоритма Штрассена весьма неочевидная задача
- Однако их можно получить исходя из общего подхода к задаче вычисления матричного умножения
- Этот подход связан с тензорными разложениями
- Тензором будем называть многомерный массив – ествественное обобщение матриц

Рассмотрим следующую матрицу $2\times 2$

$$
\begin{bmatrix} c_{1} & c_{3} \\ c_{2} & c_{4}  \end{bmatrix} =
\begin{bmatrix} a_{1} & a_{3} \\ a_{2} & a_{4}  \end{bmatrix}
\begin{bmatrix} b_{1} & b_{3} \\ b_{2} & b_{4}  \end{bmatrix}=
\begin{bmatrix} 
a_{1}b_{1} + a_{3}b_{2} & a_{1}b_{3} + a_{3}b_{4} \\ 
a_{2}b_{1} + a_{4}b_{2} & a_{2}b_{3} + a_{4}b_{4} 
\end{bmatrix}
$$

И заметим, что каждый элемент результата можно записать в следующем виде

$$ c_k = \sum_{i=1}^4 \sum_{j=1}^4 x_{ijk} a_i b_j, \quad k=1,2,3,4 $$

$x_{ijk}$ – 3-мерный массив, состоящий из 0 и 1:

$$
\begin{split}
x_{\ :,\ :,\ 1} = 
\begin{pmatrix}
1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 \\
\end{pmatrix}
\quad
x_{\ :,\ :,\ 2} = 
\begin{pmatrix}
0 & 0 & 0 & 0 \\
1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
\end{pmatrix} \\
x_{\ :,\ :,\ 3} = 
\begin{pmatrix}
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 \\
0 & 0 & 0 & 0 \\
\end{pmatrix}
\quad
x_{\ :,\ :,\ 4} = 
\begin{pmatrix}
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 \\
0 & 0 & 0 & 1 \\
\end{pmatrix}
\end{split}
$$



#### Трилинейное разложение

Метод Штрассена можно получить если разложить тензор $x_{ijk}$ следующим образом

$$ x_{ijk} = \sum_{\alpha=1}^r u_{i\alpha} v_{j\alpha} w_{k\alpha}. $$

- Это разложение называется трилинейным тензорным разложением
- Интерпретация: разделение переменных (вспомните аналогичную интерпретация SVD!) – сумма $r$ слагаемых, каждое из которых зависит от **одного** индекса результата
- Обобщение на тензоры более высокого порядка называется CP-разложением

### Получение метода Штрассена из трилинейного разложения

- Подставим общий вид трилинейного разложения

$$ c_k = \sum_{\alpha=1}^r w_{k\alpha} \left(\sum_{i=1}^4  u_{i\alpha} a_i \right) \left( \sum_{j=1}^4 v_{j\alpha} b_j\right), \quad k=1,2,3,4. $$

- Умножение на $u_{i\alpha}$, $v_{j\alpha}$ и $w_{k\alpha}$ не требует рекурсии, так как это известные факторы трилинейного разложения, которые в результате формируют необходимые комбинации блоков матриц.
- Поэтому имеем только $r$ умножения факторов вида $\left(\sum_{i=1}^4  u_{i\alpha} a_i \right)$ $\left( \sum_{j=1}^4 v_{j\alpha} b_j\right)$
 
- Если ранг $r$ окажется равным 7, то мы получаем в точности 7 умножений и метод Штрассена

## Можно ли лучше?

## AlphaTensor: агент ищет тензорное разложение с помощью обучения с подкреплением

- Алгоритм [AlphaTensor](https://www.deepmind.com/blog/discovering-novel-algorithms-with-alphatensor) показал, как подход обучения с подкреплением может помочь в поиске новых тензорных разложений
- Состояние в данном случае – это тензор
- Действие - вычитание тензора ранга 1
- Цель агента – получить нулевой тензор за минимальное количество шагов
- За каждый новый шаг получаем награду -1
- Отдельная награда даётся, если после $R_{\max}$ шагов не удалось получить нулевой тензор
- Элементы факторов берутся из множества $\{ \pm 2, \pm 1, 0 \}$

<img src="alpha_tensor_results.png" width=500>

## Выводы про ускорение матричного умножения

- Алгоритма за $O(n^2)$ не обнаружено
- Алгоритм Штрассена уменьшает асимптотическую сложность наивного метода
- Связь поиска эффективного метода матричного умножения и трилинейного тензорного разложения

## Переходим к линейным системам и LU разложению

## Линейные системы

$$ Ax = f, $$

где матрица $A$ и вектор $f$ известны.

Задача решения системы линейных уравнений – одна из основных задач вычислительной линейной алгебры.

Она возникает при решении следующих задач:

- задача линейной регрессии
- решение уравнений в частных производных и интегральных уравнений
- задачи нелинейной регрессии
- задачи оптимизации (методы Ньютона-Рафсона и Гаусса-Ньютона, условия ККТ)

## Пере- и недоопределённые линейные системы

Если система $Au = f$ имеет
- больше уравнений, чем неизвестных, она называется **переопределённой** (в общем случае не имеет решений)

- меньше уравнений, чем неизвестных, она называется **недоопределённой** (решение неединственно, нужны дополнительные предположения, чтобы гарантировать единственность решения)

## Существование решений 

Решение системы линейных уравнений с квадратной матрицей $A$

$$A u = f$$

существует тогда и только тогда, когда 
* $\det A \ne 0$

или

* матрица $A$ имеет полный ранг.

## Шкала размерностей линейных систем 

В различных приложениях размерности линейных систем могут быть различны 

- Малая: $n \leq 10^4$ (вся матрица помещается в память, **плотные матрицы**)
- Средняя: $n = 10^4 - 10^6$ (обычно **разреженные** или **структурированные** матрицы)
- Большая: $n = 10^8 - 10^9$ (обычно **разреженные** матрицы и параллельные вычисления)

## Линейные системы могут быть структурированы

- Хранение $N^2$ элементов матрицы невозможно уже для $N = 100000$.  

**Q:** как работать с такими матрицами?  

**A:** к счастью, такие матрицы чаще всего являются **структурированными** и требуют хранения $\mathcal{O}(N)$ элементов.

- Наиболее растространённый тип структурированных матриц – это разреженные матрицы: такие матрицы имеют только $\mathcal{O}(N)$ ненулевых элементов!  

- Пример (одна из самых известных матриц для $n = 5$):

$$
  \begin{pmatrix}
  2 & -1 & 0 & 0 & 0 \\
  -1 & 2 & -1 & 0 & 0 \\
  0 & -1 & 2 & -1 & 0 \\
  0 & 0 &-1& 2 & -1  \\
  0 & 0 & 0 & -1 & 2 \\
  \end{pmatrix}
$$

- По крайней мере можно хранить такие матрицы
- Также можно умножать такие матрицы на вектор быстро
- Но как решать линейные системы с такими матрицами?

## Основные вопросы о линейных системах

1. Какую точность мы можем получить от решения (из-за ошибок округления)?
2. Как мы вычислим решение? (LU разложение, метод Гаусса)
3. Какая сложность решения системы линейных уравнений?

## Как решать линейные системы?

**Важно**: забудьте о детерминантах и правиле Крамера (хотя они полезны для матриц $2 \times 2$)!

In [140]:
import numpy as np

def bad_cramer(A, b):
    det = np.linalg.det(A)
    x = np.empty_like(b)
    for i in range(b.shape[0]):
        cur_col = A[:, i].copy()
        A[:, i] = b.copy()
        cur_det = np.linalg.det(A)
        x[i] = cur_det / det
        A[:, i] = cur_col.copy()
    return x

n = 100
A = np.random.randn(n, n)
Q, _ = np.linalg.qr(A)
eigvals = np.arange(1, n+1, dtype=float)
eigvals[-1] = 10000
eigvals[0] = 0.0001
A = Q @ np.diag(eigvals) @ Q.T
b = np.random.randn(n)
# print(np.linalg.cond(A))

x_good = np.linalg.solve(A, b)
x_bad = bad_cramer(A, b)
print(np.linalg.norm(A @ x_bad - b) / np.linalg.norm(b))
print(np.linalg.norm(A @ x_good - b) / np.linalg.norm(b))
%timeit np.linalg.solve(A, b)
%timeit bad_cramer(A, b)

4.816169839849095e-07
1.1053265659484817e-09
477 µs ± 101 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
49.6 ms ± 9.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Как решать линейные системы?

Основной инструмент – исключение переменных. 

\begin{align*}
    &2 y + 3 x = 5 &\longrightarrow \quad &x = 5/3 -  2/3 y &\longrightarrow \quad & x = 5/3 -  2/3 y\\
    &2 x + 3z = 5 &\longrightarrow\quad &2(5/3 -  2/3 y) + 3z = 5 &\longrightarrow \quad & y = (-5 + 10/3 + 3z) \cdot 3/4\\
    &z + y = 2 &\longrightarrow\quad  & z + y = 2, &\longrightarrow \quad & (-5 + 10/3 + 3z) \cdot 3/4 + z = 2\\
\end{align*}

и так вы можете найти $z$ (и все остальные неизвестные).  

Этот процесс называется **методов Гаусса** и является одним из самых часто используемых алгоритмов. 

## Метод Гаусса

Метод Гаусса состоит из двух этапов:
1. Проход вниз
2. Проход вверх

## Проход вниз

- Исключим $x_1$:

$$
   x_1 = f_1 - (a_{12} x_2 + \ldots + a_{1n} x_n)/a_{11},
$$

и подставим в уравнения $2, \ldots, n$. 

- Затем исключим $x_2$ и подставим в остальные уравнения.

- Важно, что ведущий элемент (pivot), тот на который мы делим, не равен $0$.

## Проход вверх

Во время прохода назад:
- решаем уравнение для $x_n$
- подставляем решение в уравнение для $x_{n-1}$ и так далее, пока не вычислим все $x_i, i=1,\ldots, n$.

## Метод Гаусса и LU разложение

Метод Гаусса связан с вычислением одного из самых важных матричных разложений: **LU разложения**.

**Определение**: LU разложение матрицы $A$ – это представление

$$A =  LU,$$

где $L$ – **нижнетреугольная** и $U$ – **верхнетреугольная** матрица.

Это разложение **неединственно**, поэтому обычно требуют дополнительно, что на диагонали матрицы $L$ стоят 1.

## Обратная матрица: определение

Матрица, обратная к матрице $A$, это такая матрица $X$ что  

$$
   AX = XA = I, 
$$

где $I$ – единичная матрица. Обратная матрица обозначается как $A^{-1}$.

Вычисление обратной матрицы связано с решением линейной системы. В самом деле, $i$-ый столбец произведения даёт

$$
A x_i = e_i,
$$

где $e_i$ – $i$-ый столбец единичной матрицы. 
Таким образом, мы можем использовать метод Гаусса, чтобы решить эту систему.


## Решение системы линейных уравнений после вычисления LU разложения

**Основная цель** вычисления LU разложения – это решение системы линейных уравнений, поскольку

$$
    x^* = A^{-1} f = (L U)^{-1} f = U^{-1} L^{-1} f, 
$$

и задача сводится к решению двух линейных систем с верхне- и нижнетреугольными матрицами.

Проход вниз выражается в виде

$$
     L y = f, 
$$

аналогично для прохода вверх

$$
   U x = y.
$$

Всегда ли существует $LU$ разложение?

## Существование LU разложения

Алгоритм вычисления LU разложения работает, если **мы не делим на $0$** на каждом шаге метода Гаусса.

**Q:** Для какого класса матриц это так?

**A:** Это так для **строго регулярных матриц**.

## Строго регулярные матрицы и LU разложение

**Определение.** Матрица $A$ называется *строго регулярной*, если все лидирующие главные миноры (подматрицы из первых $k$ строк и $k$ столбцов) не вырождены. 

В этом случае LU разложение всегда существует. Обратное также верно (проверьте!).

## LU разложение для положительно определённых Эрмитовых матриц (разложение Холецкого)

Строго регулярные матрицы имеют LU разложение. 

Важный класс строго регулярных матриц – это класс **Эрмитовых положительно определённых матриц**

**Определение.** Матрица $A$ называется Эрмитовой положительно определённой, если для любого $x \in \mathbb{C}^n: \Vert x \Vert \ne 0$ выполнено

$$
(x, Ax) > 0.
$$

**Утверждение:** Эрмитова положительно определённая матрица $A$ строго регулярна и имеет разложение Холецкого вида

$$A = RR^*,$$

где $R$ нижнетреугольная матрица.

Часто матрица $R$ называется "квадратным корнем" матрицы $A$. 

## Вычисление LU разложения

- Во многих случаях достаточно один раз вычислить LU разложение!

- Если такое разложение найдено (что требует $\mathcal{O}(n^3)$ операций), тогда решение линейной системы сводится к решению линейных систем с матрицами $L$ и $U$, которые требуют $\mathcal{O}(n^2)$ операций.

**Упражнение:** Решение линейной системы с треугольной матрицей вычисляется быстро. Как вычислить $L$ и $U$?

## Два способа вычисления LU разложения

- Способ 1
    - Элементарными преобразованиями $E_i$ приводим матрицу $A$ к верхнетреугольному виду:
    
    $$ E_n \ldots E_1 A = U \quad A = (E_1 \ldots E_n)^{-1} U = LU$$
    
    - Так как $E_i$ – нижнетреугольные матрицы, то их произведение и обратная к их произведению матрица также нижнетреугольная, которую можно обозначить $L$

- Способ 2 (алгоритм Дулитла)
    - Распишем поэлементно равенство $A = LU$
    $$ a_{11} = u_{11} \quad a_{12} = u_{12} \quad a_{13} = u_{13}$$
    
    $$ l_{21} = \frac{a_{21}}{a_{11}} \quad l_{31} = \frac{a_{31}}{a_{11}}$$
    
    $$ u_{22} = \ldots \quad u_{23} = \ldots $$
    
    - Можно последовательно выразить строки $U$ и столбцы $L$

## Сложность метода Гаусса/LU разложения

- Каждый шаг исключения занимает $\mathcal{O}(n^2)$ операций. 

- Таким образом, сложность алгоритма $\mathcal{O}(n^3)$.  

In [143]:
def elementary_lu(A):
    n = A.shape[0]
    U = A.copy()
    L = np.eye(n)
    for i in range(n):
        el_transform = U[i+1:, i] / U[i, i]
        L[i+1:, i] = el_transform
        U[i+1:, i:] -= np.outer(L[i+1:, i], U[i, i:])
    return L, U

def doolittle(A):
    n = A.shape[0]
    U = np.zeros((n, n))
    L = np.eye(n)
    for i in range(n):
        U[i, i:] = A[i, i:] - L[i, :i] @ U[:i, i:]
        L[(i+1):, i] = (A[(i + 1):, i] - L[(i+1):] @ U[:, i]) / U[i, i]
    return L, U

n = 100
A = np.random.randn(n, n)
Q, _ = np.linalg.qr(A)
eigvals = np.arange(1, n+1, dtype=float)
eigvals[-1] = 100
eigvals[0] = 1
A = Q @ np.diag(eigvals) @ Q.T

L_e, U_e = elementary_lu(A)
L_d, U_d = doolittle(A)
print(np.linalg.norm(A - L_e @ U_e))
print(np.linalg.norm(A - L_d @ U_d))

%timeit elementary_lu(A)
%timeit doolittle(A)

1.6816004769691313e-13
2.9709118060706874e-14
1.96 ms ± 194 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
The slowest run took 10.52 times longer than the fastest. This could mean that an intermediate result is being cached.
4.19 ms ± 5.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Промежуточный итог

- Что такое линйеные системы и какими они бывают
- LU разложние и строго регулярные матрицы
- Разложение Холецкого как частный случай
- Способы вычисления и их сложность